In [1]:
##Nice Work  kirtan
import os
import cv2
from PIL import Image
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from math import ceil, sqrt
import natsort
import glob
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Disable GPU
tf.config.set_visible_devices([], 'GPU')

# tf.config.set_visible_devices([], 'GPU')  # Hide GPU devices



2023-07-31 16:27:29.361076: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 16:27:30.044071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(
2023-07-31 16:27:31.665995: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344

In [2]:
# tf.debugging.set_log_device_placement(True)

# # Create some tensors and perform an operation
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)

# print(c)
# tf.debugging.set_log_device_placement(False)


In [3]:
seed_constant = 42
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [4]:
frameDir_path = '/home/kirtan/Documents/Sign_Language/data/Frames/'
checkpoint_path = '/home/kirtan/Documents/Sign_Language/'

In [5]:
def load_json(path):
    with open(path, "r") as f:
        json_file = json.load(f)
    return json_file
label_map = load_json('./label_map_temp_include.json')
print(label_map)


{'loud': 0, 'quiet': 1, 'happy': 2, 'sad': 3, 'beautiful': 4, 'ugly': 5, 'deaf': 6, 'blind': 7, 'nice': 8, 'mean': 9, 'rich': 10, 'poor': 11, 'thick': 12, 'thin': 13, 'expensive': 14, 'cheap': 15, 'flat': 16, 'curved': 17, 'male': 18, 'female': 19, 'tight': 20, 'loose': 21, 'high': 22, 'low': 23, 'soft': 24, 'hard': 25, 'deep': 26, 'shallow': 27, 'clean': 28, 'dirty': 29, 'strong': 30, 'weak': 31, 'dead': 32, 'alive': 33, 'heavy': 34, 'light': 35, 'famous': 36, 'long': 37, 'short': 38, 'tall': 39, 'wide': 40, 'narrow': 41, 'biglarge': 42, 'smalllittle': 43, 'slow': 44, 'fast': 45, 'hot': 46, 'cold': 47, 'warm': 48, 'cool': 49, 'new': 50, 'old': 51, 'young': 52, 'good': 53, 'bad': 54, 'wet': 55, 'dry': 56, 'sick': 57, 'healthy': 58}


In [6]:
labels_train = []
filenames_train = []
masks_train =[]
labels_test = []
filenames_test = []
masks_test =[]
labels_val = []
filenames_val = []
masks_val =[]
seq_length=80
def get_Lables(mode):
    train_label = []
    train_filename = []
    train_mask=[]
    train_split_file = f'./train_test_split/temp_include_{mode}.txt'
    train_file = open(train_split_file, 'r')
    for line in train_file:
        label = "".join([i for i in line if i.isalpha()]).lower()
        label = label[10:]
        label = label[:-6]
        line = line.split("/")
        line.pop(1)
        line.insert(1,label)
        last_word = line[-1].strip('\n')
        last_word = last_word + "_frames"
        line[-1] = last_word
        line = "/".join(line)
        line = frameDir_path + line #frame folder
        frame_names = os.listdir(line)
        frame_names = natsort.natsorted(frame_names)
        orginal_length = len(frame_names)
        padding_length = seq_length - orginal_length
        for i in range(orginal_length):
            temp_fileName= os.path.join(line, frame_names[i])
            train_filename.append(temp_fileName)
        train_filename = train_filename + [""] * padding_length
    for frame in train_filename:
        if frame != "":
            split_frame = frame.split("/")
            train_label.append(split_frame[-3])
            train_mask.append(True)  # Valid element, set mask to True
        else:
            train_label.append(-1)
            train_mask.append(False)  # Padded element, set mask to False
    for i in range(len(train_label)):
        if train_label[i] != -1:
            train_label[i] = label_map[train_label[i]]
    train_file.close()
    return np.array(train_filename),np.array(train_label),np.array(train_mask)

filenames_train,labels_train,masks_train = get_Lables('train')
filenames_val,labels_val,masks_val = get_Lables('val')
# filenames_test,labels_test,masks_test = get_Lables('test')

In [7]:
# list_temp =[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
# print(len(list_temp))

In [8]:
print('filenames_train.shape',filenames_train.shape)
print("labels_train.shape",labels_train.shape)
print("masks_train.shape",masks_train.shape)
# print("filenames_test.shape",filenames_test.shape)
# print("labels_test.shape",labels_test.shape)
# print("masks_test.shape",masks_test.shape)
print("filenames_val.shape",filenames_val.shape)
print("labels_val.shape",labels_val.shape)
print("masks_val.shape",masks_val.shape)

filenames_train.shape (46077,)
labels_train.shape (46077,)
masks_train.shape (46077,)
filenames_val.shape (5299,)
labels_val.shape (5299,)
masks_val.shape (5299,)


In [9]:
def parse_function(filename, label):
    if filename != "":
        image = tf.io.read_file(filename)
        print(image.shape)
        image = tf.image.decode_jpeg(image)
        print(image.shape)
        image = tf.image.convert_image_dtype(image, tf.float32)
        print(image.shape)
        image = tf.image.resize(image, [64, 64])
        print(image.shape)
        image = tf.ensure_shape(image, (64, 64, 3))
        print(image.shape)
    else:
        image = tf.zeros([64, 64,3], dtype=tf.float32)
    label = tf.one_hot(label, 59)
    label = tf.cast(label, tf.int64)  # Convert labels to int64
    return image, label
def train_preprocess(image, label):
    if tf.reduce_all(image != 0):
        image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, 0.15)
        image = tf.image.random_contrast(image, 0.8, 1.5)
        image = tf.image.random_saturation(image, 0.6, 3)
    return image, label

In [10]:
batch_size = 80
padded_shapes = ([64, 64, 3],[59])
# create train dataset
dataset_train = tf.data.Dataset.from_tensor_slices((filenames_train, labels_train))
dataset_train = dataset_train.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
dataset_train = dataset_train.map(train_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_train = dataset_train.batch(80,drop_remainder=True)
dataset_train = dataset_train.batch(8,drop_remainder=True)

# create validation dataset
dataset_val = tf.data.Dataset.from_tensor_slices((filenames_val, labels_val))
dataset_val = dataset_val.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
dataset_val = dataset_val.map(train_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset_val = dataset_val.batch(80,drop_remainder=True)
dataset_val = dataset_val.batch(8,drop_remainder=True)


()
(None, None, None)
(None, None, None)
(64, 64, None)
(64, 64, 3)
()
(None, None, None)
(None, None, None)
(64, 64, None)
(64, 64, 3)


In [11]:
for batch in dataset_train:
    images, labels = batch
    print('image_len',len(images),images.shape)
    print('labels_len',len(labels),labels.shape)
    for image in images:
        for img in image:
            print(img.shape)
#     print('batch',batch)
    # print(len(batch))
    break
print(len(dataset_train))
print(dataset_val)

2023-07-31 16:27:32.970046: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [46077]
	 [[{{node Placeholder/_0}}]]
2023-07-31 16:27:32.970728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [46077]
	 [[{{node Placeholder/_1}}]]


image_len 8 (8, 80, 64, 64, 3)
labels_len 8 (8, 80, 59)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64,

KeyboardInterrupt: 

In [ ]:
# def testparse_function(filename,label):
#     image = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(image)
#     image = tf.image.convert_image_dtype(image, tf.float32)
#     image = tf.image.resize(image, [64, 64])
#     image = tf.cast(image, tf.float32)
#     label = tf.one_hot(label, 3)
#     return image,label
# def testtrain_preprocess(image,label):
#     image = tf.image.random_brightness(image, 0.15)
#     image = tf.image.random_contrast(image, 0.8, 1.5)
#     image = tf.image.random_saturation(image, 0.6, 3)
#     return image,label
# padded_shapes = ([64, 64, 3],[3])
# # create test dataset
# dataset_test = tf.data.Dataset.from_tensor_slices((filenames_test,labels_test))
# dataset_test = dataset_test.map(testparse_function, num_parallel_calls=tf.data.AUTOTUNE)
# # dataset_test = dataset_test.map(testtrain_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
# dataset_test = dataset_test.padded_batch(batch_size, padded_shapes=padded_shapes,drop_remainder=True)
# dataset_test = dataset_test.batch(3,drop_remainder=True)
# dataset_test = dataset_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
#### from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ReLU,TimeDistributed,Masking, Conv2D, ReLU, Dropout, Concatenate,MaxPooling2D, BatchNormalization, LSTM, GlobalAveragePooling2D, Dense
from tensorflow.keras.activations import gelu
from tensorflow.keras.models import Sequential

# class CRNN():
#     def __init__(self, sample_size=64, sample_duration=80, num_classes=59,
#                  lstm_hidden_size=512, lstm_num_layers=1):
#         super(CRNN, self).__init__()
#         print(x)
#         self.sample_size = sample_size
#         self.sample_duration = sample_duration
#         self.num_classes = num_classes

#         # network params
#         self.ch1, self.ch2, self.ch3, self.ch4 = 64, 128, 256, 512
#         self.k1, self.k2, self.k3, self.k4 = (2, 2), (2, 2), (2, 2), (2, 2)
#         self.s1, self.s2, self.s3, self.s4 = (2, 2), (1, 1), (1, 1), (1, 1)
#         self.p1, self.p2, self.p3, self.p4 = (0, 0), (0, 0), (0, 0), (0, 0)
#         self.d1, self.d2, self.d3, self.d4 = (1, 1), (1, 1), (1, 1), (1, 1)
#         self.lstm_input_size = self.ch4
#         self.lstm_hidden_size = lstm_hidden_size
#         self.lstm_num_layers = lstm_num_layers

#         # network architecture
#         self.conv1 = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=self.ch1, kernel_size=self.k1, strides=self.s1, padding='valid', dilation_rate=self.d1,input_shape=(None,80,3,64,64)),
#             tf.keras.layers.BatchNormalization(momentum=0.01),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(filters=self.ch1, kernel_size=1, strides=1),
#             tf.keras.layers.MaxPool2D(pool_size=2,name='max_pooling2d_1'),
#         ])
#         self.conv2 = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=self.ch2, kernel_size=self.k2, strides=self.s2, padding='valid', dilation_rate=self.d2),
#             tf.keras.layers.BatchNormalization(momentum=0.01),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(filters=self.ch2, kernel_size=1, strides=1),
#             tf.keras.layers.MaxPool2D(pool_size=2,name='max_pooling2d_2'),
#         ])
#         self.conv3 = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=self.ch3, kernel_size=self.k3, strides=self.s3, padding='valid', dilation_rate=self.d3),
#             tf.keras.layers.BatchNormalization(momentum=0.01),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(filters=self.ch3, kernel_size=1, strides=1),
#             tf.keras.layers.MaxPool2D(pool_size=2,name='max_pooling2d_3'),
#         ])
#         self.conv4 = tf.keras.Sequential([
#             tf.keras.layers.Conv2D(filters=self.ch4, kernel_size=self.k4, strides=self.s4, padding='valid', dilation_rate=self.d4),
#             tf.keras.layers.BatchNormalization(momentum=0.01),
#             tf.keras.layers.ReLU(),
#             tf.keras.layers.Conv2D(filters=self.ch4, kernel_size=1, strides=1),
#             tf.keras.layers.GlobalAvgPool2D(name='gloabl_pooling2d_1'),
#         ])
#         self.lstm = tf.keras.layers.LSTM(units=self.lstm_hidden_size, return_sequences=True,stateful=True)
#         self.fc1 = tf.keras.layers.Dense(units=self.num_classes)

#     def call(self, x):
#         cnn_embed_seq = []
#         print(x)
#         for t in range(x.shape[2]):
#             out = self.conv1(x[:, :, t, :, :])
#             out = self.conv2(out)
#             out = self.conv3(out)
#             out = self.conv4(out)

#             out = tf.reshape(out, (out.shape[0], -1))
#             cnn_embed_seq.append(out)

#         cnn_embed_seq = tf.stack(cnn_embed_seq, axis=0)
#         cnn_embed_seq = tf.transpose(cnn_embed_seq, perm=[1, 0, 2])

#         self.lstm.reset_states()
#         out = self.lstm(cnn_embed_seq)
#         out = self.fc1(out[:, -1, :])

#         return out
    

def create_LRCN_model(input_shape=(80, 64, 64, 3), num_classes=59):
    model = Sequential()

    model.add(TimeDistributed(Conv2D(64, (3, 3),strides=(2,2), padding="valid", activation='gelu',
                                     use_bias=False, input_shape=input_shape,name="conv1")))
    model.add(TimeDistributed(BatchNormalization(momentum=0.01)))
        
    model.add(TimeDistributed(ReLU()))
    
    model.add(TimeDistributed(Conv2D(64, (1, 1),strides=(1,1), activation='gelu',
                                     use_bias=False,name="conv2")))
    
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    
    ##Block 2
    
    
    model.add(TimeDistributed(Conv2D(128, (3, 3),strides=(1,1), padding="valid", activation='gelu',
                                     use_bias=False,name="conv3")))
    model.add(TimeDistributed(BatchNormalization(momentum=0.01)))

    model.add(TimeDistributed(ReLU()))
    
    model.add(TimeDistributed(Conv2D(128, (1, 1),strides=(1,1), activation='gelu',
                                     use_bias=False,name="conv4")))
        
    model.add(TimeDistributed(MaxPooling2D((2, 2))))    
    
    ##Block 3
    
    
    model.add(TimeDistributed(Conv2D(256, (3, 3),strides=(1,1),padding="valid", activation='gelu',
                                     use_bias=False,name="conv5")))
    model.add(TimeDistributed(BatchNormalization(momentum=0.01)))

    model.add(TimeDistributed(ReLU()))
    
    model.add(TimeDistributed(Conv2D(256, (1, 1),strides=(1,1), activation='gelu',
                                     use_bias=False,name="conv6")))
        
    model.add(TimeDistributed(MaxPooling2D((2, 2)))) 
        
        
    
    ##Block 4
    
    model.add(TimeDistributed(Conv2D(256, (2, 2),strides=(1,1),padding="valid", activation='gelu',
                                     use_bias=False,name="conv7")))
    model.add(TimeDistributed(BatchNormalization(momentum=0.01)))

    model.add(TimeDistributed(ReLU()))
    
    model.add(TimeDistributed(Conv2D(256, (1, 1),strides=(1,1), activation='gelu',
                                     use_bias=False,name="conv8")))
        
    model.add(TimeDistributed(GlobalAveragePooling2D())) 
    
    ## LSTM 
    
    model.add(TimeDistributed(Reshape((-1, 256))))  # Reshape to (time_steps, features)

    model.add(TimeDistributed(LSTM(256, return_sequences=True)))
    
    model.add(TimeDistributed(Dense(256, activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(1e-3))))

    # Final dense layer for classification
    model.add(TimeDistributed(Dense(num_classes,activation=gelu, kernel_regularizer=tf.keras.regularizers.l2(1e-3))))

    return model

    

In [ ]:
print(tf.__version__)

2.12.0


In [ ]:
LRCN_model = create_LRCN_model()

In [ ]:
# tf.debugging.set_log_device_placement(False)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow logging level to suppress messages
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True, monitor='val_loss', verbose=1, 
                                                  save_best_only=True, mode='min')
LRCN_model.compile(loss=tf.keras.losses.categorical_crossentropy,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   metrics=[tf.keras.metrics.CategoricalAccuracy(),
                            tf.keras.metrics.RecallAtPrecision(precision=0.9, name='recallAtPrecision'),
                            tf.keras.metrics.Recall(),
                            tf.keras.metrics.Precision(),
                            ])
hist = LRCN_model.fit(x = dataset_train,validation_data=dataset_val,verbose=1, epochs = 50,callbacks = [cp_callback,early_stopping_callback])


Epoch 1/50


2023-06-08 16:31:31.256391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 16:31:31.259086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 16:31:31.261147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

ValueError: in user code:

    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/kirtan/.conda/envs/crossway/lib/python3.9/site-packages/keras/en

In [ ]:
folder_path = "/home/kirtan/Documents/Sign_Language/data/Frames/Adjectives/dry/MVI_5167.MOV_frames"
frame_pathList = []
image_sequence=[]
for i in range(40):
    temp_path = folder_path + f'/frame_{i}.jpg'
    frame_pathList.append(temp_path)
    temp_path = ""

for path in frame_pathList:
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [64, 64])
    image_sequence.append(image)
desired_length = 80
padding_length = desired_length - len(image_sequence)

if padding_length > 0:
    padding_images = tf.zeros([padding_length, 64, 64, 3], dtype=tf.float32)
    image_sequence = tf.concat([image_sequence, padding_images], axis=0)

image_sequence = tf.stack(image_sequence, axis=0)  # Create the image sequence
image_sequence = tf.expand_dims(image_sequence, axis=0)
print(image_sequence.shape) 
predictions = LRCN_model.predict(image_sequence)
print(predictions.shape)

(1, 80, 64, 64, 3)
1/1 [==============================] - 0s 150ms/step
(1, 80, 59)


In [ ]:
# print("Evaluate on test data")
# results = LRCN_model.evaluate(dataset_test)
# print("test loss, test acc:", results)

print("Evaluate on train data")
results = LRCN_model.evaluate(dataset_train)
print("train loss, trai acc:", results)

print("Evaluate on validation data")
results = LRCN_model.evaluate(dataset_val)
print("val loss, val acc:", results)

In [ ]:
print(predictions)
predicted_labels = np.argmax(predictions, axis=2)
print(predicted_labels)
final = np.max(predicted_labels)
print(final)

[[[0.0246892  0.03619457 0.04104808 ... 0.05684435 0.02114256 0.04000509]
  [0.02373577 0.03230406 0.04580003 ... 0.05955273 0.0194433  0.04120204]
  [0.02351937 0.03051795 0.04802994 ... 0.06088576 0.01844789 0.04187407]
  ...
  [0.01146932 0.01168903 0.02472704 ... 0.00239041 0.01938326 0.00646256]
  [0.01106928 0.01096132 0.02407236 ... 0.00230767 0.01911121 0.0065612 ]
  [0.01011055 0.01008899 0.0239671  ... 0.00205243 0.0185706  0.00601373]]]
[[45 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53
  53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53  2  2 57 57 57 44 44 44
  44 44 44 44 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 34 20 20
  20 20 20 20 20 20 20 20]]
57


In [ ]:
# With `dilation_rate` as 2.
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(
    2, 3,
    activation='relu',
    dilation_rate=2)(x)
print(y.shape)



(4, 24, 24, 2)
